## Northwinds Sales Data Warehouse
### Python Script 

#### 1) Load libraries and configure Snowflake connector

In [2]:
import snowflake.connector
import pandas as pd
import numpy as np
import json
from datetime import datetime, date
from Support_Functions import create_database_table
from Support_Functions import create_initial_surrogate_key_mapping_table

In [5]:
with open('credentials.json') as f:
    data = json.load(f)
    user = data['user']
    password = data['password']
    account = data['account']

In [9]:
# Connect to Snowflake
connection = snowflake.connector.connect(
    user = user,
    password = password,
    account = account
)
cur = connection.cursor()

#### 2)  Create and Populate the Star Schema

In [9]:
def create_dataframe(sql, surrogate_key, start_date = datetime(2000,1,1), end_date = datetime(2099,12,31)):
    cur.execute(sql)
    rows = cur.fetchall()
    columns = [column[0] for column in cur.description]
    dimension = pd.DataFrame(rows, columns = columns)
    dimension, mapping = create_initial_surrogate_key_mapping_table(dimension, surrogate_key, start_date, end_date)
    
    return dimension, mapping

In [10]:
def create_table(table, mapping, surrogate_key, table_name):
    database = 'northwinds_sales_data_warehouse'
    mapping_name = table_name + '_surrogate_mapping_table'
    create_database_table(table, surrogate_key, database, table_name)
    create_database_table(mapping, surrogate_key, database, mapping_name)

##### Product Dimension Table

In [6]:
product_sql = 'SELECT p.ProductID AS Product_ID, p.ProductName, c.CategoryName \
                FROM INTERNATIONAL_FOODS.public.Products p \
                LEFT JOIN INTERNATIONAL_FOODS.public.Categories c \
                USING(CategoryID)'

product, product_surrogate_key_mapping = create_dataframe(product_sql, 'Product_Key')

In [27]:
create_table(product, product_surrogate_key_mapping, 'Product_Key', 'Product')

In [7]:
product

,Product_Key,PRODUCT_ID,PRODUCTNAME,CATEGORYNAME
0,0,1,Chai,Beverages
1,1,2,Chang,Beverages
2,2,3,Aniseed Syrup,Condiments
3,3,4,Chef Antons Cajun Seasoning,Condiments
4,4,5,Chef Antons Gumbo Mix,Condiments
...,...,...,...,...
72,72,73,R d Kaviar,Seafood
73,73,74,Longlife Tofu,Produce
74,74,75,Rh nbr u Klosterbier,Beverages
75,75,76,Lakkalik ri,Beverages


In [8]:
product_surrogate_key_mapping

,Product_Key,PRODUCT_ID,Start_Date,End_Date,Current_Flag
0,0,1,2000-01-01,2099-12-31,True
1,1,2,2000-01-01,2099-12-31,True
2,2,3,2000-01-01,2099-12-31,True
3,3,4,2000-01-01,2099-12-31,True
4,4,5,2000-01-01,2099-12-31,True
...,...,...,...,...,...
72,72,73,2000-01-01,2099-12-31,True
73,73,74,2000-01-01,2099-12-31,True
74,74,75,2000-01-01,2099-12-31,True
75,75,76,2000-01-01,2099-12-31,True


##### Customer Dimension Table

In [12]:
customer_sql = 'SELECT CustomerID AS Customer_ID, CompanyName AS Customer_Name, Country AS Customer_Country \
                FROM INTERNATIONAL_FOODS.public.Customers'

customer, customer_surrogate_key_mapping = create_dataframe(customer_sql, 'Customer_Key')

In [28]:
create_table(customer, customer_surrogate_key_mapping, 'Customer_Key', 'Customer')

In [13]:
customer

,Customer_Key,CUSTOMER_ID,CUSTOMER_NAME,CUSTOMER_COUNTRY
0,0,ALFKI,Alfreds Futterkiste,Germany
1,1,ANATR,Ana Trujillo Emparedados y helados,Mexico
2,2,ANTON,Antonio Moreno Taquer a,Mexico
3,3,AROUT,Around the Horn,UK
4,4,BERGS,Berglunds snabbk p,Sweden
...,...,...,...,...
86,86,WARTH,Wartian Herkku,Finland
87,87,WELLI,Wellington Importadora,Brazil
88,88,WHITC,White Clover Markets,USA
89,89,WILMK,Wilman Kala,Finland


In [14]:
customer_surrogate_key_mapping

,Customer_Key,CUSTOMER_ID,Start_Date,End_Date,Current_Flag
0,0,ALFKI,2000-01-01,2099-12-31,True
1,1,ANATR,2000-01-01,2099-12-31,True
2,2,ANTON,2000-01-01,2099-12-31,True
3,3,AROUT,2000-01-01,2099-12-31,True
4,4,BERGS,2000-01-01,2099-12-31,True
...,...,...,...,...,...
86,86,WARTH,2000-01-01,2099-12-31,True
87,87,WELLI,2000-01-01,2099-12-31,True
88,88,WHITC,2000-01-01,2099-12-31,True
89,89,WILMK,2000-01-01,2099-12-31,True


##### Shipper Dimension Table

In [11]:
shipper_sql = 'SELECT ShipperID AS Shipper_ID, CompanyName AS Shipper_Name \
                FROM INTERNATIONAL_FOODS.public.Shippers'

shipper, shipper_surrogate_key_mapping = create_dataframe(shipper_sql, 'Shipper_Key')

In [31]:
create_table(shipper, shipper_surrogate_key_mapping, 'Shipper_Key', 'Shipper')

In [12]:
shipper

,Shipper_Key,SHIPPER_ID,SHIPPER_NAME
0,0,1,Speedy Express
1,1,2,United Package
2,2,3,Federal Shipping


In [13]:
shipper_surrogate_key_mapping

,Shipper_Key,SHIPPER_ID,Start_Date,End_Date,Current_Flag
0,0,1,2000-01-01,2099-12-31,True
1,1,2,2000-01-01,2099-12-31,True
2,2,3,2000-01-01,2099-12-31,True


##### Employee Dimension Table

In [14]:
employee_sql = 'SELECT EmployeeID AS Employee_ID, LastName, FirstName, City, Country \
                FROM INTERNATIONAL_FOODS.public.Employees'

employee, employee_surrogate_key_mapping = create_dataframe(employee_sql, 'Employee_Key')

In [34]:
create_table(employee, employee_surrogate_key_mapping, 'Employee_Key', 'Employee')

In [32]:
employee

,Employee_Key,EMPLOYEE_ID,LASTNAME,FIRSTNAME,CITY,COUNTRY
0,0,2,Fuller,Andrew,Tacoma,USA
1,1,1,Davolio,Nancy,Seattle,USA
2,2,3,Leverling,Janet,Kirkland,USA
3,3,4,Peacock,Margaret,Redmond,USA
4,4,5,Buchanan,Steven,London,UK
5,5,6,Suyama,Michael,London,UK
6,6,7,King,Robert,London,UK
7,7,8,Callahan,Laura,Seattle,USA
8,8,9,Dodsworth,Anne,London,UK


In [33]:
employee_surrogate_key_mapping

,Employee_Key,EMPLOYEE_ID,Start_Date,End_Date,Current_Flag
0,0,2,2000-01-01,2099-12-31,True
1,1,1,2000-01-01,2099-12-31,True
2,2,3,2000-01-01,2099-12-31,True
3,3,4,2000-01-01,2099-12-31,True
4,4,5,2000-01-01,2099-12-31,True
5,5,6,2000-01-01,2099-12-31,True
6,6,7,2000-01-01,2099-12-31,True
7,7,8,2000-01-01,2099-12-31,True
8,8,9,2000-01-01,2099-12-31,True


##### Date Dimension Table

In [15]:
cur.execute("SELECT OrderDate AS date \
            FROM INTERNATIONAL_FOODS.public.Orders")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
Dates = pd.DataFrame(rows, columns = columns)

Dates['DATE'] = pd.to_datetime(Dates['DATE'])

Dates['day_of_week'] = Dates['DATE'].dt.day_name()
Dates['week'] = Dates['DATE'].dt.isocalendar().week
Dates['month'] = Dates['DATE'].dt.month
Dates['year'] = Dates['DATE'].dt.year
Dates = Dates.drop_duplicates()

In [27]:
create_database_table(Dates, 'DATE', 'northwinds_sales_data_warehouse', 'Date')

In [16]:
Dates

,DATE,day_of_week,week,month,year
0,1996-07-04,Thursday,27,7,1996
1,1996-07-05,Friday,27,7,1996
2,1996-07-08,Monday,28,7,1996
4,1996-07-09,Tuesday,28,7,1996
5,1996-07-10,Wednesday,28,7,1996
...,...,...,...,...,...
812,1998-04-30,Thursday,18,4,1998
816,1998-05-01,Friday,18,5,1998
819,1998-05-04,Monday,19,5,1998
822,1998-05-05,Tuesday,19,5,1998


##### Fact Table

In [28]:
cur.execute("SELECT o.orderid, o.OrderDate AS Date, c.customerid, e.employeeid, s.shipperid, \
            p.productid, od.unitprice, od.quantity, od.discount AS DiscountAmount, \
            (od.UnitPrice * od.Quantity - od.Discount) AS TotalRevenue \
            FROM INTERNATIONAL_FOODS.public.orders o \
            LEFT JOIN INTERNATIONAL_FOODS.public.customers c USING(customerid) \
            LEFT JOIN INTERNATIONAL_FOODS.public.employees e USING(employeeid) \
            LEFT JOIN INTERNATIONAL_FOODS.public.shippers s ON s.shipperid = o.shipvia \
            LEFT JOIN INTERNATIONAL_FOODS.public.orderdetails od USING(orderid) \
            LEFT JOIN INTERNATIONAL_FOODS.public.products p USING(productid) \
            ORDER BY o.orderid")

rows = cur.fetchall()
columns = [column[0] for column in cur.description]
Initial_Fact_Table = pd.DataFrame(rows, columns = columns)
Initial_Fact_Table

,ORDERID,DATE,CUSTOMERID,EMPLOYEEID,SHIPPERID,PRODUCTID,UNITPRICE,QUANTITY,DISCOUNTAMOUNT,TOTALREVENUE
0,10248,1996-07-04,VINET,5,3,42,10,10,0,100
1,10248,1996-07-04,VINET,5,3,11,14,12,0,168
2,10248,1996-07-04,VINET,5,3,72,35,5,0,175
3,10249,1996-07-05,TOMSP,6,1,14,19,9,0,171
4,10249,1996-07-05,TOMSP,6,1,51,42,40,0,1680
...,...,...,...,...,...,...,...,...,...,...
2150,11077,1998-05-06,RATTC,1,2,7,30,1,0,30
2151,11077,1998-05-06,RATTC,1,2,6,25,1,0,25
2152,11077,1998-05-06,RATTC,1,2,4,22,1,0,22
2153,11077,1998-05-06,RATTC,1,2,3,10,4,0,40


In [29]:
#  Read the surrogate key mapping tables
cur.execute("Select * FROM NORTHWINDS_SALES_DATA_WAREHOUSE.public.CUSTOMER_SURROGATE_MAPPING_TABLE")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
customer_surrogate_mapping_table = pd.DataFrame(rows, columns = columns)

cur.execute("Select * FROM NORTHWINDS_SALES_DATA_WAREHOUSE.public.EMPLOYEE_SURROGATE_MAPPING_TABLE")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
employee_surrogate_mapping_table = pd.DataFrame(rows, columns = columns)

cur.execute("Select * FROM NORTHWINDS_SALES_DATA_WAREHOUSE.public.PRODUCT_SURROGATE_MAPPING_TABLE")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
product_surrogate_mapping_table = pd.DataFrame(rows, columns = columns)

cur.execute("Select * FROM NORTHWINDS_SALES_DATA_WAREHOUSE.public.SHIPPER_SURROGATE_MAPPING_TABLE")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
shipper_surrogate_mapping_table = pd.DataFrame(rows, columns = columns)

In [30]:
product_surrogate_mapping_table

,PRODUCT_KEY,PRODUCT_ID,START_DATE,END_DATE,CURRENT_FLAG
0,0,1,2000-01-01,2099-12-31,True
1,1,2,2000-01-01,2099-12-31,True
2,2,3,2000-01-01,2099-12-31,True
3,3,4,2000-01-01,2099-12-31,True
4,4,5,2000-01-01,2099-12-31,True
...,...,...,...,...,...
72,72,73,2000-01-01,2099-12-31,True
73,73,74,2000-01-01,2099-12-31,True
74,74,75,2000-01-01,2099-12-31,True
75,75,76,2000-01-01,2099-12-31,True


In [31]:
Sales_Fact_Table = Initial_Fact_Table
Sales_Fact_Table['EMPLOYEE_KEY'] = Sales_Fact_Table['EMPLOYEEID'].map(
    employee_surrogate_mapping_table.set_index('EMPLOYEE_ID')['EMPLOYEE_KEY'])
Sales_Fact_Table['CUSTOMER_KEY'] = Sales_Fact_Table['CUSTOMERID'].map(
    customer_surrogate_mapping_table.set_index('CUSTOMER_ID')['CUSTOMER_KEY'])
Sales_Fact_Table['PRODUCT_KEY'] = Sales_Fact_Table['PRODUCTID'].map(
    product_surrogate_mapping_table.set_index('PRODUCT_ID')['PRODUCT_KEY'])
Sales_Fact_Table['SHIPPER_KEY'] = Sales_Fact_Table['SHIPPERID'].map(
    shipper_surrogate_mapping_table.set_index('SHIPPER_ID')['SHIPPER_KEY'])
Sales_Fact_Table

,ORDERID,DATE,CUSTOMERID,EMPLOYEEID,SHIPPERID,PRODUCTID,UNITPRICE,QUANTITY,DISCOUNTAMOUNT,TOTALREVENUE,EMPLOYEE_KEY,CUSTOMER_KEY,PRODUCT_KEY,SHIPPER_KEY
0,10248,1996-07-04,VINET,5,3,42,10,10,0,100,4,84,41,2
1,10248,1996-07-04,VINET,5,3,11,14,12,0,168,4,84,10,2
2,10248,1996-07-04,VINET,5,3,72,35,5,0,175,4,84,71,2
3,10249,1996-07-05,TOMSP,6,1,14,19,9,0,171,5,78,13,0
4,10249,1996-07-05,TOMSP,6,1,51,42,40,0,1680,5,78,50,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2150,11077,1998-05-06,RATTC,1,2,7,30,1,0,30,1,64,6,1
2151,11077,1998-05-06,RATTC,1,2,6,25,1,0,25,1,64,5,1
2152,11077,1998-05-06,RATTC,1,2,4,22,1,0,22,1,64,3,1
2153,11077,1998-05-06,RATTC,1,2,3,10,4,0,40,1,64,2,1


In [33]:
Sales_Fact_Table['CUSTOMER_KEY'] = Sales_Fact_Table['CUSTOMERID']. \
    map(customer_surrogate_mapping_table.set_index('CUSTOMER_ID')['CUSTOMER_KEY'])

Sales_Fact_Table['PRODUCT_KEY'] = Sales_Fact_Table['PRODUCTID']. \
    map(product_surrogate_mapping_table.set_index('PRODUCT_ID')['PRODUCT_KEY'])

Sales_Fact_Table['EMPLOYEE_KEY'] = Sales_Fact_Table['EMPLOYEEID']. \
    map(employee_surrogate_mapping_table.set_index('EMPLOYEE_ID')['EMPLOYEE_KEY'])

Sales_Fact_Table['SHIPPER_KEY'] = Sales_Fact_Table['SHIPPERID']. \
    map(shipper_surrogate_mapping_table.set_index('SHIPPER_ID')['SHIPPER_KEY'])
Sales_Fact_Table[['SHIPPERID', 'SHIPPER_KEY']]

,PRODUCTID,PRODUCT_KEY
0,42,41
1,11,10
2,72,71
3,14,13
4,51,50
...,...,...
2150,7,6
2151,6,5
2152,4,3
2153,3,2


In [39]:
Sales_Fact_Table['DATE'] = pd.to_datetime(Sales_Fact_Table['DATE'])

dtype('<M8[ns]')

In [40]:
# Create a composite primary key by concatenating Order ID and Product ID
# for the mapping table to map Sales Key onto
Sales_Fact_Table['Natural_Key'] = Sales_Fact_Table['ORDERID'].astype(str).copy()
    + Sales_Fact_Table['PRODUCTID'].astype(str).copy()
Sales_Fact_Table = Sales_Fact_Table[['Natural_Key', 'DATE', 'PRODUCT_KEY', 'CUSTOMER_KEY', 
                                     'EMPLOYEE_KEY', 'SHIPPER_KEY', 'ORDERID', 'PRODUCTID', 
                                     'UNITPRICE', 'QUANTITY', 'DISCOUNTAMOUNT', 'TOTALREVENUE']]
Sales_Fact_Table

,Natural_Key,DATE,PRODUCT_KEY,CUSTOMER_KEY,EMPLOYEE_KEY,SHIPPER_KEY,ORDERID,PRODUCTID,UNITPRICE,QUANTITY,DISCOUNTAMOUNT,TOTALREVENUE
0,1024842,1996-07-04,41,84,4,2,10248,42,10,10,0,100
1,1024811,1996-07-04,10,84,4,2,10248,11,14,12,0,168
2,1024872,1996-07-04,71,84,4,2,10248,72,35,5,0,175
3,1024914,1996-07-05,13,78,5,0,10249,14,19,9,0,171
4,1024951,1996-07-05,50,78,5,0,10249,51,42,40,0,1680
...,...,...,...,...,...,...,...,...,...,...,...,...
2150,110777,1998-05-06,6,64,1,1,11077,7,30,1,0,30
2151,110776,1998-05-06,5,64,1,1,11077,6,25,1,0,25
2152,110774,1998-05-06,3,64,1,1,11077,4,22,1,0,22
2153,110773,1998-05-06,2,64,1,1,11077,3,10,4,0,40


In [ ]:
Sales_Fact_Table, sale_surrogate_key_mapping = \
    create_initial_surrogate_key_mapping_table(Sales_Fact_Table,
                                             'Sales_Key',
                                             datetime(2000,1,1),
                                             datetime(2099,12,31))

In [48]:
create_table(Sales_Fact_Table, sale_surrogate_key_mapping, 'Sales_Key', 'Sales_Fact')

In [41]:
Sales_Fact_Table

,Sales_Key,Natural_Key,DATE,PRODUCT_KEY,CUSTOMER_KEY,EMPLOYEE_KEY,SHIPPER_KEY,ORDERID,PRODUCTID,UNITPRICE,QUANTITY,DISCOUNTAMOUNT,TOTALREVENUE
0,0,1024842,1996-07-04,41,84,4,2,10248,42,10,10,0,100
1,1,1024811,1996-07-04,10,84,4,2,10248,11,14,12,0,168
2,2,1024872,1996-07-04,71,84,4,2,10248,72,35,5,0,175
3,3,1024914,1996-07-05,13,78,5,0,10249,14,19,9,0,171
4,4,1024951,1996-07-05,50,78,5,0,10249,51,42,40,0,1680
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2150,2150,110777,1998-05-06,6,64,1,1,11077,7,30,1,0,30
2151,2151,110776,1998-05-06,5,64,1,1,11077,6,25,1,0,25
2152,2152,110774,1998-05-06,3,64,1,1,11077,4,22,1,0,22
2153,2153,110773,1998-05-06,2,64,1,1,11077,3,10,4,0,40


In [44]:
sale_surrogate_key_mapping

,Sales_Key,Natural_Key,Start_Date,End_Date,Current_Flag
0,0,1024842,2000-01-01,2099-12-31,True
1,1,1024811,2000-01-01,2099-12-31,True
2,2,1024872,2000-01-01,2099-12-31,True
3,3,1024914,2000-01-01,2099-12-31,True
4,4,1024951,2000-01-01,2099-12-31,True
...,...,...,...,...,...
2150,2150,110777,2000-01-01,2099-12-31,True
2151,2151,110776,2000-01-01,2099-12-31,True
2152,2152,110774,2000-01-01,2099-12-31,True
2153,2153,110773,2000-01-01,2099-12-31,True


#### 3) Test Queries
1. What was the total revenue for Tofu during the month of July 1996?

In [17]:
cur.execute("Select * FROM NORTHWINDS_SALES_DATA_WAREHOUSE.public.DATE")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
date = pd.DataFrame(rows, columns = columns)

cur.execute("Select * FROM NORTHWINDS_SALES_DATA_WAREHOUSE.public.EMPLOYEE")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
employee = pd.DataFrame(rows, columns = columns)

cur.execute("Select * FROM NORTHWINDS_SALES_DATA_WAREHOUSE.public.PRODUCT")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
product = pd.DataFrame(rows, columns = columns)

cur.execute("Select * FROM NORTHWINDS_SALES_DATA_WAREHOUSE.public.Sales_Fact")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
fact = pd.DataFrame(rows, columns = columns)

In [19]:
datamart1 = (fact
           .merge(product, on = 'PRODUCT_KEY', how = 'left')
           .merge(date, on = 'DATE', how = 'left'))
datamart1

,SALES_KEY,NATURAL_KEY,DATE,PRODUCT_KEY,CUSTOMER_KEY,EMPLOYEE_KEY,SHIPPER_KEY,ORDERID,PRODUCTID,UNITPRICE,QUANTITY,DISCOUNTAMOUNT,TOTALREVENUE,PRODUCT_ID,PRODUCTNAME,CATEGORYNAME,DAY_OF_WEEK,WEEK,MONTH,YEAR
0,0,1024842,1996-07-04,41,84,4,2,10248,42,10,10,0,100,42,Singaporean Hokkien Fried Mee,Grains/Cereals,Thursday,27,7,1996
1,1,1024811,1996-07-04,10,84,4,2,10248,11,14,12,0,168,11,Queso Cabrales,Dairy Products,Thursday,27,7,1996
2,2,1024872,1996-07-04,71,84,4,2,10248,72,35,5,0,175,72,Mozzarella di Giovanni,Dairy Products,Thursday,27,7,1996
3,3,1024914,1996-07-05,13,78,5,0,10249,14,19,9,0,171,14,Tofu,Produce,Friday,27,7,1996
4,4,1024951,1996-07-05,50,78,5,0,10249,51,42,40,0,1680,51,Manjimup Dried Apples,Produce,Friday,27,7,1996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2150,2150,110777,1998-05-06,6,64,1,1,11077,7,30,1,0,30,7,Uncle Bobs Organic Dried Pears,Produce,Wednesday,19,5,1998
2151,2151,110776,1998-05-06,5,64,1,1,11077,6,25,1,0,25,6,Grandmas Boysenberry Spread,Condiments,Wednesday,19,5,1998
2152,2152,110774,1998-05-06,3,64,1,1,11077,4,22,1,0,22,4,Chef Antons Cajun Seasoning,Condiments,Wednesday,19,5,1998
2153,2153,110773,1998-05-06,2,64,1,1,11077,3,10,4,0,40,3,Aniseed Syrup,Condiments,Wednesday,19,5,1998


In [21]:
revenue = datamart1[(datamart1.PRODUCTNAME == 'Tofu') & (datamart1.MONTH == 7) 
                    & (datamart1.YEAR == 1996)]['TOTALREVENUE'].sum()
print(f'The total revenue for Tofu during July, 1996 is {revenue} dollars.')

The total revenue for Tofu during July, 1996 is 171 dollars.


In [15]:
cur.execute("SELECT SUM(TOTALREVENUE) AS TOTAL_REVENUE \
            FROM NORTHWINDS_SALES_DATA_WAREHOUSE.public.Sales_Fact \
            LEFT JOIN NORTHWINDS_SALES_DATA_WAREHOUSE.public.Product USING(PRODUCT_KEY) \
            LEFT JOIN NORTHWINDS_SALES_DATA_WAREHOUSE.public.Date USING(DATE) \
            WHERE PRODUCTNAME = 'Tofu' AND MONTH = 7 AND YEAR = 1996")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
df_3A = pd.DataFrame(rows, columns = columns)
df_3A

,TOTAL_REVENUE
0,171


2. Find the top performing employee in each country based on the total value of orders processed.

In [19]:
sql_3b = \
"WITH EmployeeRank AS ( \
    SELECT CONCAT(FirstName, ' ', LastName) AS Name, COUNTRY, SUM(UnitPrice * Quantity - DiscountAmount) AS Order_Value, \
        RANK() OVER (PARTITION BY COUNTRY ORDER BY SUM(UnitPrice * Quantity - DiscountAmount) DESC) AS \
        RankPerCountry \
    FROM NORTHWINDS_SALES_DATA_WAREHOUSE.public.Sales_Fact \
    LEFT JOIN NORTHWINDS_SALES_DATA_WAREHOUSE.public.Employee USING(EMPLOYEE_KEY) \
    GROUP BY COUNTRY, NAME) \
SELECT NAME, COUNTRY, Order_Value \
FROM EmployeeRank \
WHERE RankPerCountry = 1"
cur.execute(sql_3b)
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
df_3B = pd.DataFrame(rows, columns = columns)
df_3B

,NAME,COUNTRY,ORDER_VALUE
0,Robert King,UK,141489
1,Margaret Peacock,USA,250451


In [23]:
datamart2 = (fact.merge(employee, on = 'EMPLOYEE_KEY', how = 'left'))

datamart2['NAME'] = datamart2['FIRSTNAME'] + ' ' + datamart2['LASTNAME']
order_value = datamart2.groupby(['COUNTRY', 'NAME'])['TOTALREVENUE'].sum()
uk = pd.DataFrame(order_value[order_value == order_value.groupby('COUNTRY').max()[0]])
us = pd.DataFrame(order_value[order_value == order_value.groupby('COUNTRY').max()[1]])

top_employees = pd.concat([uk, us])
top_employees

,,TOTALREVENUE
COUNTRY,NAME,
UK,Robert King,141489
USA,Margaret Peacock,250451
